In [47]:
import graphviz as gv

In [48]:
'''The next thing on the todo list is to make different colors/styles for extinct or extant species'''

'The next thing on the todo list is to make different colors/styles for extinct or extant species'

In [49]:
#We'll use functools to make things a little quicker.
import functools
graph = functools.partial(gv.Graph, format='svg')
digraph = functools.partial(gv.Digraph, format='svg')

In [50]:
style = {
    'graph': {
        'label': 'Tree of Life',
        'labelloc' : 't',
        'fontsize': '16',
        'fontcolor': 'black',
        'bgcolor': 'white',
        'rankdir': 'TB',
    },
    'nodes': {
        'fontname': 'Helvetica',
        'shape': 'hexagon',
        'fontcolor': 'black',
        'color': 'white',
        'style': 'filled',
        'fillcolor': 'green',
    },
    'edges': {
        'style': 'dashed',
        'color': 'black',
        'arrowhead': 'open',
        'fontname': 'Courier',
        'fontsize': '12',
        'fontcolor': 'white',
    }
}

In [51]:
#Here we'll build some helper functions
def add_nodes(graph, nodes):
    for n in nodes:
        if isinstance(n, tuple):
            graph.node(n[0], **n[1])
        else:
            graph.node(n)
    return graph

def add_edges(graph, edges):
    for e in edges:
        if isinstance(e[0], tuple):
            graph.edge(*e[0], **e[1])
        else:
            graph.edge(*e)
    return graph

def apply_style(graph, style):
    graph.graph_attr.update(
        ('graph' in style and style['graph']) or {}
    )
    graph.node_attr.update(
        ('nodes' in style and style['nodes']) or {}
    )
    graph.edge_attr.update(
        ('edges' in style and style['edges']) or {}
    )
    return graph


In [83]:
precambrian = ('Pre-cambrian life', None, 600) # Confirmed
insects = ('Insects', precambrian[0], 480) # Confirmed
vertebrates = ('Vertebrates', precambrian[0], 525) # Confirmed
sharks = ('Sharks', vertebrates[0], 450) # Confirmed
tetrapods = ('Tetrapods', vertebrates[0], 340)
amniotes = ('Amniotes', tetrapods[0], 322)
amphibians = ('Amphibians', tetrapods[0], 333)
sauropsids = ('Sauropsids', amniotes[0], 300)
synapsids = ('Synapsids', amniotes[0], 180)
archosaurs = ('Archosaurs', sauropsids[0], 240)
therian = ('Therian mammals', synapsids[0], 160)

In [84]:
#Make a list of all nodes in the form [name, parent, date in millions of years ago] #Make them tuples, although they could be lists
all_nodes = [precambrian, vertebrates, insects, sharks, tetrapods, amniotes, amphibians, sauropsids, archosaurs,
             synapsids, therian]

In [85]:
def node_manager(date, nodes=all_nodes, parent=precambrian):
    '''This function allows you to find all the nodes (and their respective edges) given a particular date.
    
    Date is in millions of years ago.'''
    #Find the date of the common ancestor for this clade
    keeper_nodes=[]
    edges = []
    #for x in range(len(nodes)):
        
    for node in nodes:
        if node[2] > date and node[2] < parent[2]: #if the node's date is older than the date selected, but
            #younder than the parent node, add it to the keeper list
            keeper_nodes.append(node[0])
            print('node {}'.format(node))
            print('node1 {}'.format(node[1]))
            print('node0 {}'.format(node[0]))
            #try:
            #    print(node[0] + "', {'xlabel': " + str(node[2]))
            #except TypeError:
            #    print("Node {} is giving you a NoneType".format(node))
            #Then, for the edges, you'll want a tuple in the form (parent, self)
            try:
                #print(node[1] + node[0])
                edges.append(tuple([node[1], node[0]]))
            except TypeError:
                print("Edge {} is giving you a NoneType".format(node))    
    return (keeper_nodes, edges)

In [86]:
nodes, edges = node_manager(300)

node ('Vertebrates', 'Pre-cambrian life', 525)
node1 Pre-cambrian life
node0 Vertebrates
node ('Insects', 'Pre-cambrian life', 480)
node1 Pre-cambrian life
node0 Insects
node ('Sharks', 'Vertebrates', 450)
node1 Vertebrates
node0 Sharks
node ('Tetrapods', 'Vertebrates', 340)
node1 Vertebrates
node0 Tetrapods
node ('Amniotes', 'Tetrapods', 322)
node1 Tetrapods
node0 Amniotes
node ('Amphibians', 'Tetrapods', 333)
node1 Tetrapods
node0 Amphibians


In [87]:
treeoflife = add_edges(
    add_nodes(digraph(), nodes),
    edges
)

In [88]:
treeoflife = apply_style(treeoflife, style)
treeoflife.render('img/tol')

'img/tol.svg'

In [82]:
all_nodes

[('Pre-cambrian life', None, 600),
 ('Vertebrates', ('Pre-cambrian life', None, 600), 525),
 ('Insects', ('Pre-cambrian life', None, 600), 480),
 ('Sharks', ('Vertebrates', ('Pre-cambrian life', None, 600), 525), 450),
 ('Tetrapods', ('Vertebrates', ('Pre-cambrian life', None, 600), 525), 340),
 ('Amniotes',
  ('Tetrapods', ('Vertebrates', ('Pre-cambrian life', None, 600), 525), 340),
  322),
 ('Amphibians',
  ('Tetrapods', ('Vertebrates', ('Pre-cambrian life', None, 600), 525), 340),
  333),
 ('Sauropsids',
  ('Amniotes',
   ('Tetrapods', ('Vertebrates', ('Pre-cambrian life', None, 600), 525), 340),
   322),
  300),
 ('Archosaurs',
  ('Sauropsids',
   ('Amniotes',
    ('Tetrapods', ('Vertebrates', ('Pre-cambrian life', None, 600), 525), 340),
    322),
   300),
  240),
 ('Synapsids',
  ('Amniotes',
   ('Tetrapods', ('Vertebrates', ('Pre-cambrian life', None, 600), 525), 340),
   322),
  180),
 ('Therian mammals',
  ('Synapsids',
   ('Amniotes',
    ('Tetrapods', ('Vertebrates', ('Pre-